<a href="https://colab.research.google.com/github/aso1801009/AI_teach2020/blob/master/2020AI0402_Morphological.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 形態素解析（MorphologicalAnalysis)

### 形態素解析ライブラリMeCabのインストール

In [ ]:
# 形態素解析ライブラリMeCabと辞書（meCab-ipadic-NEologd）のインストール
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!pip install mecab-python3 > /dev/null

# MeCabの実行時の指定パスをインストールパスにリンクさせる
# シンボリックリンク（/etc/mecabrc を /usr/local/etc/mecabrc で参照できるようにする）
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

### まずMecabのデフォルト辞書で文章を解析する


In [ ]:
# インストールしたMeCabをimport
import MeCab
# MeCabオブジェクトの生成（デフォルト辞書で）
tagger = MeCab.Tagger()
# 形態素解析を実行させる
result = tagger.parse("メイが恋ダンスを踊っている。")
# 解析結果を表示
print( result )

メイ	名詞,一般,*,*,*,*,*
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
恋	名詞,一般,*,*,*,*,恋,コイ,コイ
ダンス	名詞,サ変接続,*,*,*,*,ダンス,ダンス,ダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
。	記号,句点,*,*,*,*,。,。,。
EOS



###最新の情報が入った辞書（ipadic-neologd)で形態素解析してみる

In [ ]:
# github上からipadic-neologdの最新版をダウンロード
! git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null
# ipadic-neologd のインストール
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.



辞書のインストール済みディレクトリを表示確認

In [ ]:
# 辞書のインストール済みディレクトリの場所を表示
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


###辞書(ipadic-neologd)の場所を指定してMeCabで形態素解析

In [ ]:
import MeCab
# mecab-ipadic-NEologd辞書を指定して、MeCabオブジェクトの生成
path_neologd = "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
tagger = MeCab.Tagger( "-d " + path_neologd )
# オブジェクトインスタンスで形態素解析を行う
result = tagger.parse( "メイが恋ダンスを踊っている。" )
print( result )

メイ	名詞,固有名詞,人名,一般,*,*,M.A.Y,メイ,メイ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
恋ダンス	名詞,固有名詞,一般,*,*,*,恋ダンス,コイダンス,コイダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
。	記号,句点,*,*,*,*,。,。,。
EOS



###形態素解析結果を品詞分析して名詞、動詞、形容詞だけを抜き出すことでストップワードを除外する

In [ ]:
import MeCab
# mecab-ipadic-NEologd辞書を指定して、MeCabオブジェクトの生成
path_neologd = "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
tagger = MeCab.Tagger( "-d" + path_neologd )
# 形態素解析して、結果（リスト）を取得
node = tagger.parseToNode("メイが恋ダンスを踊っている。")
print( node )

result = [] # 抽出結果を登録する配列
# リストのリンクNextがNoneになるまでループ処理
while node is not None:
  # 品詞情報取得 node詳細の先頭（品詞情報）だけを抜き出す
  hinshi = node.feature.split(",")[0]
  if hinshi in ["名詞"]:
    # 名詞だったら、表層系を結果配列 result に追加
    result.append( node.surface )
  elif hinshi in ["動詞", "形容詞"]:
    # 動詞、形容詞だったら、node形態素情報から原形情報(index6番目)をresult配列に追加
    result.append( node.feature.split(",")[6] )
  # if/elif文終わり
  node = node.next
# whileループ終わり
print( result )

<Swig Object of type 'MeCab::Node *' at 0x7f56d09f4c70>
['メイ', '恋ダンス', '踊る', 'いる']
